In [1]:
import pandas as pd
import eclabfiles as ecf
from matplotlib import pyplot as plt
import numpy as np
import glob
from datetime import datetime
from pickle_utils import read_ecf, read_emsi
%matplotlib qt

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
DATE = '2024-03-14'

In [4]:
folder = 'T:\\Team\\Szewczyk\\Data\\' + DATE
ecfile = glob.glob(folder+'\\*.mpr')[-1]
datfiles = glob.glob(folder+'\\*.dat')

In [5]:
ecfdf, ecf_start = read_ecf(ecfile)

In [28]:
joined_emsi = pd.DataFrame([])
for filename in datfiles:
    emsidf, emsi_start, emsi_end = read_emsi(filename)
    if emsi_end < ecf_start:
        continue
    emsidf.t = emsidf.t - ecf_start + emsi_start
    joined_emsi = pd.concat([joined_emsi, emsidf], ignore_index=True)

For now let's skip the data alignment – if needed, it can be added here.

In [31]:
ecfdf['emsi'] = np.interp(ecfdf.t, joined_emsi.t, joined_emsi.emsi)
ecfdf['light'] = np.interp(ecfdf.t, joined_emsi.t, joined_emsi.light)

In [35]:
ecfdf.to_pickle(folder+'\\data.pkl')